# Collaborative Filtering dan Tren Analisis Penjualan (Prediksi dan Rekomendasi)

In [1]:
pip install pandas scikit-learn matplotlib seaborn openpyxl lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\user1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
# --- 0. Instalasi & Import Library ---
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import warnings
import time

# Mengabaikan pesan peringatan agar output lebih bersih
warnings.filterwarnings('ignore')

# --- 1. Memuat & Membersihkan Data ---
try:
    file_path = 'Sistem Rekomendasi Abon - Dataset Abon Ummi.csv'
    df = pd.read_csv(file_path, on_bad_lines='skip')
    print("✅ Data berhasil dimuat.")
except FileNotFoundError:
    print("❌ Error: File 'Sistem Rekomendasi Abon - Dataset Abon Ummi.csv' tidak ditemukan.")
    # Data dummy jika file tidak ada, agar kode tetap bisa berjalan
    data = {'Tanggal': ['01/01/2025', '01/01/2025'], 'Nama Abon': ['Abon Sapi', 'Abon Ayam'], 'Jenis Abon': ['Original', 'Pedas'], 'Berat Kemasan': ['100 gram', '100 gram'], 'Jumlah': [10, 15], 'Harga Satuan': ['Rp50.000,00', 'Rp45.000,00'], 'Total': ['Rp500.000,00', 'Rp675.000,00']}
    df = pd.DataFrame(data)
    print("ℹ️ Menggunakan data dummy sebagai contoh.")

# --- 2. Feature Engineering & Preprocessing (MAKSIMAL) ---
print("🚀 Memulai Feature Engineering & Preprocessing (MAKSIMAL)...")
df['Tanggal'] = pd.to_datetime(df['Tanggal'], format='%d/%m/%Y')
df['Harga Satuan'] = df['Harga Satuan'].astype(str).str.replace(r'[Rp.,]', '', regex=True).astype(float)
if 'Total' in df.columns:
    df = df.drop(columns=['Total'])
df = df.sort_values(by=['Tanggal', 'Nama Abon']).reset_index(drop=True)

# Membuat fitur-fitur baru dari data tanggal dan penjualan
df['Bulan'] = df['Tanggal'].dt.month
df['Tahun'] = df['Tanggal'].dt.year
df['Minggu_dalam_Tahun'] = df['Tanggal'].dt.isocalendar().week.astype(int)
df['Hari_dalam_Minggu'] = df['Tanggal'].dt.dayofweek
df['Pekan_dalam_Bulan'] = (df['Tanggal'].dt.day - 1) // 7 + 1
df['Musim_Lebaran'] = df['Bulan'].isin([3, 4]).astype(int)
df['Penjualan_Bulan_Lalu'] = df.groupby(['Nama Abon', 'Jenis Abon', 'Berat Kemasan'])['Jumlah'].shift(1).fillna(0)
df['Penjualan_2Bulan_Lalu'] = df.groupby(['Nama Abon', 'Jenis Abon', 'Berat Kemasan'])['Jumlah'].shift(2).fillna(0)
df['Penjualan_3Bulan_Lalu'] = df.groupby(['Nama Abon', 'Jenis Abon', 'Berat Kemasan'])['Jumlah'].shift(3).fillna(0)
g = df.groupby(['Nama Abon', 'Jenis Abon', 'Berat Kemasan'])['Jumlah']
df['Rata2_Penjualan_3Bulan'] = g.transform(lambda x: x.rolling(window=3, min_periods=1).mean()).fillna(0)
df['Rata2_Penjualan_6Bulan'] = g.transform(lambda x: x.rolling(window=6, min_periods=1).mean()).fillna(0)
df['Std_Penjualan_3Bulan'] = g.transform(lambda x: x.rolling(window=3, min_periods=1).std()).fillna(0)
df['Bulan_Sin'] = np.sin(2 * np.pi * df['Bulan'] / 12)
df['Bulan_Cos'] = np.cos(2 * np.pi * df['Bulan'] / 12)

# Mengubah data kategori menjadi angka
encoders = {}
for col in ['Nama Abon', 'Jenis Abon', 'Berat Kemasan']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    encoders[col] = le
print("✅ Preprocessing Maksimal Selesai.")
print("\n" + "="*60 + "\n")


# --- 3. Pelatihan Model (OPTIMAL & CEPAT) ---
print("🚀 Memulai Pelatihan Model dengan RandomizedSearchCV...")
features = [
    'Nama Abon', 'Jenis Abon', 'Berat Kemasan', 'Harga Satuan', 'Bulan', 'Tahun', 
    'Minggu_dalam_Tahun', 'Hari_dalam_Minggu', 'Pekan_dalam_Bulan', 'Musim_Lebaran',
    'Penjualan_Bulan_Lalu', 'Penjualan_2Bulan_Lalu', 'Penjualan_3Bulan_Lalu',
    'Rata2_Penjualan_3Bulan', 'Rata2_Penjualan_6Bulan', 'Std_Penjualan_3Bulan',
    'Bulan_Sin', 'Bulan_Cos'
]
X = df[features]
y = df['Jumlah']
param_dist = {
    'n_estimators': [100, 200, 300, 400], 'learning_rate': [0.03, 0.05, 0.1, 0.15],
    'num_leaves': [20, 31, 40, 50], 'max_depth': [-1, 5, 10, 15],
    'reg_alpha': [0.0, 0.1, 0.5], 'reg_lambda': [0.0, 0.1, 0.5]
}
tscv = TimeSeriesSplit(n_splits=5)
final_model_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(random_state=42), param_distributions=param_dist, 
    n_iter=40, cv=tscv, n_jobs=-1, verbose=0, 
    scoring='neg_mean_squared_error', random_state=42
)
final_model_search.fit(X, y)
best_model = final_model_search.best_estimator_

print("--- Ringkasan Evaluasi Model ---")
print(f"✅ Model FINAL terbaik ditemukan dengan parameter: {final_model_search.best_params_}")
print(f"✅ Skor (RMSE) validasi silang terbaik pada model FINAL: {np.sqrt(-final_model_search.best_score_):.2f}")
print("\n--- Alokasi Data Training & Testing (Validasi Silang) ---")
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"  -> Fold {i+1}:")
    print(f"     Data Training : {len(train_index)} sampel (indeks 0 s.d. {train_index[-1]})")
    print(f"     Data Testing  : {len(test_index)} sampel (indeks {test_index[0]} s.d. {test_index[-1]})")
print("\n" + "="*60 + "\n")


# --- 4. Prediksi Masa Depan (SANGAT OPTIMAL) ---
def predict_future_recursive_optimized(model, encoders, df_history, start_date_str, num_months):
    all_predictions = []
    dynamic_history = df_history.copy()
    produk_unik = dynamic_history[['Nama Abon', 'Jenis Abon', 'Berat Kemasan', 'Harga Satuan']].drop_duplicates().reset_index(drop=True)
    current_date = pd.to_datetime(start_date_str)
    group_keys = ['Nama Abon', 'Jenis Abon', 'Berat Kemasan']
    for i in range(num_months):
        future_df = produk_unik.copy()
        future_df['Tanggal'] = current_date
        future_df['Bulan'] = future_df['Tanggal'].dt.month
        future_df['Tahun'] = future_df['Tanggal'].dt.year
        future_df['Minggu_dalam_Tahun'] = future_df['Tanggal'].dt.isocalendar().week.astype(int)
        future_df['Hari_dalam_Minggu'] = future_df['Tanggal'].dt.dayofweek
        future_df['Pekan_dalam_Bulan'] = (future_df['Tanggal'].dt.day - 1) // 7 + 1
        future_df['Musim_Lebaran'] = future_df['Bulan'].isin([3, 4]).astype(int)
        future_df['Bulan_Sin'] = np.sin(2 * np.pi * future_df['Bulan'] / 12)
        future_df['Bulan_Cos'] = np.cos(2 * np.pi * future_df['Bulan'] / 12)
        for lag in [1, 2, 3]:
            lag_date = current_date - pd.DateOffset(months=lag)
            lag_hist = dynamic_history[(dynamic_history['Tahun'] == lag_date.year) & (dynamic_history['Bulan'] == lag_date.month)]
            lag_sales = lag_hist[group_keys + ['Jumlah']].drop_duplicates(subset=group_keys, keep='last')
            feature_name = f'Penjualan_{lag}Bulan_Lalu' if lag > 1 else 'Penjualan_Bulan_Lalu'
            lag_sales = lag_sales.rename(columns={'Jumlah': feature_name})
            future_df = pd.merge(future_df, lag_sales, on=group_keys, how='left')
        hist_3m = dynamic_history[dynamic_history['Tanggal'] >= current_date - pd.DateOffset(months=3)]
        stats_3m = hist_3m.groupby(group_keys)['Jumlah'].agg(['mean', 'std']).reset_index().rename(columns={'mean': 'Rata2_Penjualan_3Bulan', 'std': 'Std_Penjualan_3Bulan'})
        future_df = pd.merge(future_df, stats_3m, on=group_keys, how='left')
        hist_6m = dynamic_history[dynamic_history['Tanggal'] >= current_date - pd.DateOffset(months=6)]
        stats_6m = hist_6m.groupby(group_keys)['Jumlah'].agg(['mean']).reset_index().rename(columns={'mean': 'Rata2_Penjualan_6Bulan'})
        future_df = pd.merge(future_df, stats_6m, on=group_keys, how='left')
        future_df.fillna(0, inplace=True)
        X_future = future_df[features]
        future_df['Prediksi_Jumlah'] = np.round(np.maximum(0, model.predict(X_future))).astype(int)
        all_predictions.append(future_df)
        update_history_df = future_df.copy()
        update_history_df.rename(columns={'Prediksi_Jumlah': 'Jumlah'}, inplace=True)
        dynamic_history = pd.concat([dynamic_history, update_history_df[df_history.columns]], ignore_index=True)
        current_date += pd.DateOffset(months=1)
    return pd.concat(all_predictions, ignore_index=True)

print("🚀 Memulai Prediksi dengan Fungsi yang Dioptimalkan...")
start_time = time.time()
start_date = '2025-08-01'
months_to_predict = 6
predictions_df = predict_future_recursive_optimized(best_model, encoders, df, start_date, months_to_predict)
end_time = time.time()
print(f"✅ Prediksi Selesai dalam {end_time - start_time:.2f} detik.")
print("\n" + "="*60 + "\n")

# --- 5. Membuat Rekomendasi Aplikatif & Output ---
print("🚀 Membuat Rekomendasi Stok...")
def generate_recommendations(df_pred):
    df_pred['Peringkat'] = df_pred.groupby('Periode')['Prediksi_Jumlah'].rank(method='dense', ascending=False)
    quantiles_df = df_pred.groupby('Periode')['Prediksi_Jumlah'].quantile([0.25, 0.75]).unstack()
    quantiles_df.rename(columns={0.25: 'q25', 0.75: 'q75'}, inplace=True)
    df_pred_with_quantiles = pd.merge(df_pred, quantiles_df, on='Periode', how='left').fillna(0)
    conditions = [
        df_pred_with_quantiles['Prediksi_Jumlah'] >= df_pred_with_quantiles['q75'],
        (df_pred_with_quantiles['Prediksi_Jumlah'] < df_pred_with_quantiles['q75']) & (df_pred_with_quantiles['Prediksi_Jumlah'] >= df_pred_with_quantiles['q25']),
        df_pred_with_quantiles['Prediksi_Jumlah'] < df_pred_with_quantiles['q25']
    ]
    choices = ['🥇 Stok Prioritas Tinggi', '🥈 Stok Prioritas Sedang', '🥉 Stok Prioritas Rendah']
    df_pred['Rekomendasi_Stok'] = np.select(conditions, choices, default='Stok Standar')
    return df_pred[['Periode', 'Produk', 'Prediksi_Jumlah', 'Peringkat', 'Rekomendasi_Stok']]

for col, le in encoders.items():
    predictions_df[col] = le.inverse_transform(predictions_df[col])
predictions_df['Produk'] = predictions_df['Nama Abon'] + ' (' + predictions_df['Jenis Abon'] + ', ' + predictions_df['Berat Kemasan'] + ')'
predictions_df['Periode'] = predictions_df['Tanggal'].dt.strftime('%Y-%m')

recommendations = generate_recommendations(predictions_df)
output_file = 'Rekomendasi_Stok_Abon_Ummi_FINAL.xlsx'
with pd.ExcelWriter(output_file) as writer:
    recommendations.to_excel(writer, sheet_name='Semua Rekomendasi', index=False)
    for periode, data in recommendations.groupby('Periode'):
        data.sort_values(by='Peringkat').to_excel(writer, sheet_name=f'Rekomendasi_{periode}', index=False)
print(f"✅ Rekomendasi lengkap telah disimpan ke file: '{output_file}'")

# --- 6. Menampilkan Narasi Rekomendasi Prioritas di Terminal (Multi-Bulan) ---

# ⭐ --- FUNGSI YANG DIPERBARUI --- ⭐
def tampilkan_narasi_prioritas(rekomendasi_df, top_n=3):
    """Mencetak narasi untuk produk prioritas teratas untuk setiap bulan."""
    
    print("\n" + "="*60)
    print("📋 RANGKUMAN REKOMENDASI STOK PER BULAN".center(60))
    print("="*60)
    
    # Dapatkan semua periode unik dan urutkan
    semua_periode = sorted(rekomendasi_df['Periode'].unique())
    
    # Lakukan iterasi untuk setiap periode/bulan
    for periode in semua_periode:
        df_periode = rekomendasi_df[rekomendasi_df['Periode'] == periode].copy()
        
        # Konversi bulan dan tahun untuk narasi
        nama_bulan = pd.to_datetime(periode).strftime('%B %Y')
        
        # Ambil N produk teratas
        top_produk = df_periode.sort_values(by='Peringkat').head(top_n)
        
        print(f"\n--- Stok Produk Prioritas untuk Bulan {nama_bulan} ---")
        
        for i, row in top_produk.iterrows():
            try:
                nama, detail = row['Produk'].split(' (')
                rasa, berat = detail.replace(')', '').split(', ')
                print(f"{int(row['Peringkat'])}) {nama}, Rasa {rasa}, Varian {berat.capitalize()}: {row['Prediksi_Jumlah']} produk")
            except ValueError:
                print(f"{int(row['Peringkat'])}) {row['Produk']}: {row['Prediksi_Jumlah']} produk")

# Panggil fungsi untuk menampilkan narasi multi-bulan
tampilkan_narasi_prioritas(recommendations, top_n=3)

✅ Data berhasil dimuat.
🚀 Memulai Feature Engineering & Preprocessing (MAKSIMAL)...
✅ Preprocessing Maksimal Selesai.


🚀 Memulai Pelatihan Model dengan RandomizedSearchCV...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 1228, number of used features: 17
[LightGBM] [Info] Start training from score 2.654723
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi